# Experimental notebook for document structured knowledge extraction

This is the fourth iteration of the pipeline, the first one to be pushed in the repo, and thus the first official version. Version history will now be handled by git.


In [ ]:
# Standard library

import json
import time
import re

from typing import List
import csv


# Validation / data models
from pydantic import BaseModel


# RDF / graph libraries
from rdflib import Graph, Namespace, RDF, Literal
from rdflib.namespace import SH

# Visualization and SHACL
from pyshacl import validate

# Parsing helpers

import datetime

import yaml

# Local imports
from src.llm_utils import initialize_gemini_client, call_gemini, call_gemini_pdf, call_gemini_json, with_retries
from src.graph_utils import visualize_graph, get_semantic_hash, validate_shacl_syntax


### Initialization


In [ ]:

GEMINI_MODEL = "gemini-2.5-pro"

initialize_gemini_client(model_name=GEMINI_MODEL)

# Which document we'll be testing
DOCUMENT_NAME = "student_housing"

CSV_FILE = "Master_Results.csv"

current_run_id = 1 # this will be part of the framework later, fetched from the csv last entry


Aux functions.

In [ ]:

# Read txt file into a string (used for prompts)
def read_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()
    
# Read a JSON file and return it as string or raw JSON
def read_json(path, raw = False):
    with open(path, "r") as f:
        json_file = json.load(f)
    if (raw):
        return json_file
    # Else, convert JSON to string 
    return json.dumps(json_file, indent=2)






## Preparation: Get everything ready for logging


In [ ]:
# This function creates a blank slate for a single run
def initialize_run_context(run_id, doc_name, model_name):
    return {
        # --- Metadata ---
        "Run ID": run_id,
        "Document Name": doc_name,
        "Timestamp": datetime.datetime.now().isoformat(sep=" ", timespec="seconds"),
        "Model Name": model_name,
        
        # --- Pipeline Artifacts (Placeholders) ---
        "Service Graph Hash": "N/A",
        "SHACL Graph Hash": "N/A",
        "SHACL Valid Syntax": False,
        "SHACL Error Type": "N/A",
        "SHACL Error Message": "N/A",
        
        # --- Scenario Specifics (Will be overwritten per scenario) ---
        "Scenario ID": "N/A",
        "Scenario Description": "N/A",
        "Expected Violation Count": 0,
        "Actual Violation Count": 0,
        "Violated Shapes": [],
        "Violation Messages": [],
        "Raw Validation Report": "N/A",
        
        # --- Execution Stats ---
        "Execution Time": 0.0,
        "Successfully Executed": False,
    }
    
ctx = initialize_run_context(current_run_id, DOCUMENT_NAME, GEMINI_MODEL)


## Phase 1: Public service modeling


In [ ]:
# We begin
execution_start_time = time.time()

### 1.1 Document → Preconditions Summary

Use LLM to summarize the document into a list of preconditions.

In [ ]:
file_name = f"Precondition documents/{DOCUMENT_NAME}.pdf"

prompt = read_txt('Prompts/summarization.txt')

preconditions_summary = with_retries(call_gemini_pdf, prompt, file_name)
print(preconditions_summary)

# Save to a file too
with open(f"Results/{DOCUMENT_NAME} preconditions summary.txt", "w") as f:
    f.write(preconditions_summary)

Title: Student Housing Allowance
Conditions:
- The student must be a Greek citizen or a citizen of another European Union country.
- The student must be enrolled in a Higher Education Institution (HEI) and possess a valid academic ID.
- The student must have successfully passed exams in at least half of the courses of the previous academic year for which the allowance is requested.
- The annual family income for the previous tax year must not exceed €30,000. This limit is increased by €3,000 for each dependent child after the first.
- The student must rent a property in a city different from their family's primary residence due to their studies.
- Neither the student nor their parents can have full ownership or usufruct of a residence in the city of study.
- The total area of real estate (owner-occupied or rented out) owned by the student or their parents must not exceed 200 square meters, excluding properties in municipalities with a population under 3,000.
- To qualify for an increas

### 1.2. Preconditions Summary + Citizen Schema (TTL) → Information Model (JSON)



In [ ]:
preconditions_summary = read_txt(f"Results/{DOCUMENT_NAME} preconditions summary.txt")
citizen_schema = read_txt(f"Citizens/{DOCUMENT_NAME} schema.ttl")

class Paths(BaseModel):
    path: List[str]
    datatype: str
    
class InformationConcept(BaseModel):
    name: str
    related_paths: List[Paths]  # links the concept to citizen data available
    
class Constraint(BaseModel):
    name: str
    desc: str
    constrains: List[InformationConcept]  

schema = list[Constraint]

# Formulate prompt content and call Gemini
prompt = read_txt('Prompts/preconditions_to_JSON.txt')
content = [prompt, preconditions_summary, citizen_schema]

info_model = with_retries(call_gemini_json, content, schema)

# Save to a file too
with open(f"Results/{DOCUMENT_NAME} information model.json", "w") as f:
    f.write(info_model)

### 1.3 Information Model (JSON) → Public Service Graph (TTL)

Use deterministic code to turn the JSON into a knowledge graph using TTL syntax.

In [ ]:
PREFIXES = """@prefix ex: <http://example.org/> .
@prefix cccev: <http://data.europa.eu/m8g/> .
@prefix cpsv: <http://purl.org/vocab/cpsv#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

"""

# Parse JSON string
info_model = read_json(f"Results/{DOCUMENT_NAME} information model.json", raw=True)

# Get service name. As per the prompt, it's in the first line of the preconditions summary file
with open(f"Results/{DOCUMENT_NAME} preconditions summary.txt") as f:
    line = f.readline()
    service_name = re.findall(r'Title: (.+)', line)[0].strip().replace(" ", "_")

triples = [PREFIXES]
triples.append(f"ex:{service_name} a cpsv:PublicService .\n\n")

# Convert constraints + concepts into triples
for constraint in info_model:
    constraint_name = constraint["name"]
    constraint_desc = constraint["desc"].replace('"', '\\"')

    # Public service -> holdsRequirement -> constraint
    triples.append(f"ex:{service_name} cpsv:holdsRequirement ex:{constraint_name} .\n")

    # Constraint node
    triples.append(f'ex:{constraint_name} a cccev:Constraint ; dct:description "{constraint_desc}" .\n')

    # InformationConcept nodes
    for concept in constraint.get("constrains", []):
        concept_name = concept["name"]

        # Link constraint to concept
        triples.append(f"ex:{constraint_name} cccev:constrains ex:{concept_name} .\n")

        # Declare information concept
        triples.append(f'ex:{concept_name} a cccev:InformationConcept .\n')

    triples.append("\n")  # spacing for readability

triples_string = "".join(triples)
# Save to a file too
with open(f"Results/{DOCUMENT_NAME} service graph.ttl", "w") as f:
    f.write(triples_string)   
     
# Log 
ctx["Service Graph Hash"] = get_semantic_hash(triples_string)

### 1.4. Graph Visualization / Inspection

Visualize part of the knowledge graph to more easily inspect correct structure and logic.

In [ ]:
# Render graph of the public service
visualize_graph(f"Results/{DOCUMENT_NAME} service graph.ttl")

## Phase 2: SHACL Rule Generation


### 2.1. Information Model (JSON) → SHACL-spec (JSON)

Use deterministic code on the JSON from before to make a new intermediate JSON that contains only the necessary information to construct SHACL shapes, one for each constraint.

In [ ]:
# Load the information model JSON
info_model = read_json(f"Results/{DOCUMENT_NAME} information model.json", raw=True)

shacl_spec_json = []

for constraint in info_model:
    # 1. Rename for clarity downstream
    shape_name = constraint["name"].replace("_condition", "_shape")
    desc = constraint["desc"]
    
    concepts = []
    
    # 2. Iterate concepts (e.g., family_income, residency_city)
    for concept in constraint.get("constrains", []):
        related_paths = []
        
        paths_source = concept.get("related_paths", []) 
        
        for rp in paths_source:
            # Capture the path AND the datatype (URI vs Literal)
            related_paths.append({
                "path": rp["path"],
                "datatype": rp["datatype"] 
            })
            
        concepts.append({
            "name": concept["name"],
            "related_paths": related_paths
        })
    
    shacl_spec_json.append({
        "shape_name": shape_name,
        "desc": desc,
        "concepts": concepts
    })

# Save to file
with open(f"Results/{DOCUMENT_NAME} shacl-spec.json", "w") as f:
    json.dump(shacl_spec_json, f, indent=2)

### 2.2. SHACL-spec (JSON) + Citizen Schema (TTL) → SHACL Shapes (TTL)



In [ ]:
# Load JSON as string 
shacl_spec_json = read_json(f"Results/{DOCUMENT_NAME} shacl-spec.json")
citizen_schema = read_txt(f"Citizens/{DOCUMENT_NAME} schema.ttl")

prompt = read_txt('Prompts/shacl_spec_to_shacl_ttl.txt')
content = [prompt, shacl_spec_json, citizen_schema]

shacl_shapes = with_retries(call_gemini, content)

# Cleanup gemini markdown formatting
shacl_shapes = shacl_shapes.strip("`").replace("turtle", "").replace("ttl", "").strip()

# Save it to a file too
with open(f"Results/{DOCUMENT_NAME} shacl shapes.ttl", "w") as f:
    f.write(shacl_shapes)
    
# Log
ctx["SHACL Graph Hash"] = get_semantic_hash(shacl_shapes)

is_valid, error_stage, error_message = validate_shacl_syntax(shacl_shapes)
ctx["SHACL Valid Syntax"] = is_valid
ctx["SHACL Error Type"] = error_stage
ctx["SHACL Error Message"] = error_message


## Phase 3: Citizen - Service Modeling


### 3.1 Public Service Graph (TTL) + Citizen Graph (TTL) + Information Model (JSON) → Citizen-Service Graph (TTL) 

We expand the Public Service Graph to include Citizen data, properly connected with edges.

In [ ]:
EX = Namespace("http://example.org/")
SC = Namespace("http://example.org/schema#")

# Load service and citizen ttl's and info model
public_service_ttl = f"Results/{DOCUMENT_NAME} service graph.ttl"
citizen_ttl = f"Citizens/{DOCUMENT_NAME} eligible.ttl"
info_model = read_json(f"Results/{DOCUMENT_NAME} information model.json", raw=True)

# Realize them into graphs
g = Graph()
g.parse(public_service_ttl, format="turtle")
citizen_g = Graph()
citizen_g.parse(citizen_ttl, format="turtle")

# Merge citizen triples into main graph
for t in citizen_g:
    g.add(t)
    
# Automatically determine the root citizen node 
root_candidates = list(citizen_g.subjects(predicate=None, object=SC.Applicant))
citizen_root = root_candidates[0]

# Helper: resolve node paths (return nodes, not literals) 
def resolve_node_path(citizen_g, root_uri, path_list, datatype):
    
    # 1. Determine how deep to go
    if datatype == "URI":
        # For Identity logic (City, Person), the Value IS the Node.
        traversal_parts = path_list
    else:
        # For Value logic (Income, Area), the Value is a Literal. Stop one step BEFORE the literal to get the Node holding it.
        traversal_parts = path_list[:-1]

    # 2. Traverse
    current_nodes = {root_uri}
    
    for part in traversal_parts:
        next_nodes = set()
        pred = SC[part] # Assumes our schema matches the namespace
        
        for node in current_nodes:
            # Find all objects connected by this predicate
            for obj in citizen_g.objects(node, pred):
                # Safety check: Ensure we don't accidentally traverse into a Literal 
                # (unless it's the final step of a URI path, but usually URIs point to URIs)
                if isinstance(obj, Literal) and datatype == "URI":
                     continue # Skip weird data errors
                next_nodes.add(obj)
        
        current_nodes = next_nodes
        
        # Optimization: If dead end, stop early
        if not current_nodes:
            return set()

    return current_nodes

# Add mapsTo edges  
for constraint in info_model:
    for concept in constraint["constrains"]:
        concept_uri = EX[concept["name"]]

        for path_obj in concept["related_paths"]: 
            path_list = path_obj["path"]
            dtype = path_obj["datatype"] 
            
            # Pass the datatype to the resolver
            subject_nodes = resolve_node_path(citizen_g, citizen_root, path_list, dtype)

            for subj in subject_nodes:
                # Connect the Information Concept to the Data Node
                g.add((concept_uri, EX.mapsTo, subj))

# Serialize unified graph into ttl and save to file
g.serialize(f"Results/{DOCUMENT_NAME} citizen-service graph.ttl", format="turtle")

<Graph identifier=N1b3b3494b4634305b57f0f1232329d47 (<class 'rdflib.graph.Graph'>)>

### 3.2 Visualize the unified graph

We reuse the same function from before.

In [ ]:
visualize_graph(f"Results/{DOCUMENT_NAME} citizen-service graph.ttl")

In [ ]:
# This marks the end of the main pipeline. 
ctx["Execution Time"] = round(time.time() - execution_start_time)


## Phase 4: Citizen Validation and Scenarios

### SHACL Shape Validation

Use deterministic code and a pre-made synthetic citizen graph, constructed to fulfil all conditions, and check it against the shape. If something went wrong, a list of violations and comments will be printed.

In [14]:
# Helper to parse SHACL validation report
def parse_validation_report(conforms, results_graph, results_text):
    """
    Parses the SHACL validation graph into a flat dictionary for CSV logging.
    """
    # If it passed, return a clean success record
    if conforms:
        return {
            "violation_count": 0,
            "failed_shapes": "None",
            "messages": "None",
            "full_report": "Conforms: True"
        }

    # If it failed, extract details from the graph
    violations = []
    failed_shapes = set()
    messages = []

    # Find all nodes of type sh:ValidationResult
    for result_node in results_graph.subjects(RDF.type, SH.ValidationResult):
        
        # Extract the Shape Name (Source Shape) should return a URI like http://example.org/income_shape
        source_shape = results_graph.value(result_node, SH.sourceShape)
        if source_shape:
            # Split to get just "income_shape"
            shape_name = str(source_shape).split("/")[-1].split("#")[-1]
            failed_shapes.add(shape_name)

        # Extract the Message
        message = results_graph.value(result_node, SH.resultMessage)
        if message:
            messages.append(str(message))

    return {
        "violation_count": len(messages),
        "failed_shapes": "; ".join(sorted(list(failed_shapes))), # Stringify for CSV
        "messages": " | ".join(messages), # Stringify for CSV
        "full_report": results_text # Keep raw text just in case
    }
    

In [ ]:

def apply_mutations(base_graph, actions):
    """
    Applies text-based Turtle mutations.
    """
    
    # Pre-defined header to allow "lazy" Turtle snippets in YAML
    PREFIX_HEADER = """
    @prefix : <http://example.org/schema#> .
    @prefix ex: <http://example.org/> .
    @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
    """
    
    # 1. Clone the graph (Memory safe)
    new_graph = Graph()
    for t in base_graph: new_graph.add(t)
    
    for action in actions:
        if action['type'] == 'replace_node':
            snippet = action['turtle']
            
            # A. Parse the snippet into a temporary graph
            # We prepend prefixes so the user doesn't have to type them every time
            full_turtle = PREFIX_HEADER + "\n" + snippet
            snippet_graph = Graph()
            try:
                snippet_graph.parse(data=full_turtle, format="turtle")
            except Exception as e:
                raise ValueError(f"Invalid Turtle in mutation: {e}")

            # B. Identify the Subject(s) being replaced
            # The snippet describes 1+ nodes. We find them all.
            subjects = set(snippet_graph.subjects())
            
            for s in subjects:
                # C. Wipe OLD properties for this subject from the main graph
                # We remove (s, ?, ?) -> outgoing edges
                # We KEEP (?, ?, s) -> incoming edges (e.g. Dad -> hasIncome -> DadIncome)
                new_graph.remove((s, None, None))
                
                # D. Add NEW properties from the snippet
                for p, o in snippet_graph.predicate_objects(s):
                    new_graph.add((s, p, o))
                    
        elif action['type'] == 'no_action':
            pass
    
    new_graph.bind("ex", Namespace("http://example.org/"))     
    new_graph.bind("", Namespace("http://example.org/schema#"))
           
    return new_graph

# Write to csv
def flush_context_to_csv(context_dict, csv_file):
    # Read headers from the existing file
    with open(csv_file, 'r', encoding='utf-8', newline='') as f:
        reader = csv.reader(f)
        headers = next(reader) # Grabs the first row (the headers)

    # Align data to those headers
    row_data = []
    for h in headers:
        value = context_dict.get(h, "N/A") # safe get with default
        row_data.append(value)

    # Write the row
    with open(csv_file, 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row_data)
        


Begin the scenario loop, writing to csv after every scenario.

In [ ]:
# Load the Golden Citizen (Baseline)
golden_ttl = "Citizens/student_housing eligible.ttl"
golden_graph = Graph()
golden_graph.parse(golden_ttl, format="turtle")

# Load the Scenarios from YAML
with open("Citizens/student_housing scenarios.yaml", "r") as f:
    scenarios = yaml.safe_load(f)

# Iterate and Apply
for scn in scenarios:
    ctx["Scenario ID"] = scn['id']
    ctx["Scenario Description"] = scn['description']
    ctx["Expected Violation Count"] = scn['expected_violation_count']
    
    # Apply mutations (Returns a NEW graph, leaving golden_graph untouched)
    mutated_graph = apply_mutations(golden_graph, scn['actions'])

    # Proceed to Validation 
    conforms, results_graph, results_text = validate(
        data_graph=mutated_graph,
        shacl_graph=f"Results/{DOCUMENT_NAME} shacl shapes.ttl",
        inference='rdfs',
    )
    
    # Parse validation report
    parse_result = parse_validation_report(conforms, results_graph, results_text)
    ctx["Actual Violation Count"] = parse_result["violation_count"]
    ctx["Violated Shapes"] = parse_result["failed_shapes"]
    ctx["Violation Messages"] = parse_result["messages"]

    # If we made it this far
    ctx["Successfully Executed"] = True
    
    flush_context_to_csv(ctx, CSV_FILE)
    
# End of the run
print(f"Logged Run {ctx['Run ID']} to CSV.") 


Logged Run 1 to CSV.
